# Grado en ciencias de datos - Big Data


# Práctica 3.1 - Machine Learning con Spark

En esta práctica vamos a ver la librería Mlib de Apache Spark. Sigue todos los bloques y prueba a cambiar los valores para así comprobar su funcionamiento.

Ten en cuenta que una vez tengas en marcha Spark, podrás visualizar la evolución de cada trabajo de Spark en  <http://localhost:4040>

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("Ejemplo pySparkSQL") \
    .config("spark.sql.warehouse.dir", "file:///D:/tmp/spark-warehouse") \
    .getOrCreate()

sc = spark.sparkContext


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/11/18 10:15:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Otros imports necesarios:

In [2]:
%matplotlib inline 
import matplotlib.pyplot as plt
from test_helper import Test
from pyspark.sql.functions import *
from pyspark.sql import Row

In [3]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import LogisticRegression

# Ejemplos de código con LogisticRegression
Vamos a entrenar un modelo de regresión logística con MLlib

In [4]:
# Creamos unos datos de entrenamiento como lista de tuplas (label, features).
training = spark.createDataFrame([
    (1.0, Vectors.dense([0.0, 1.1, 0.1])),
    (0.0, Vectors.dense([2.0, 1.0, -1.0])),
    (0.0, Vectors.dense([2.0, 1.3, 1.0])),
    (1.0, Vectors.dense([0.0, 1.2, -0.5]))], ["label", "features"])

# Creamos una instancia de LogisticRegression, esta instancia es un Estimator.
lr = LogisticRegression(maxIter=10, regParam=0.01)

# Imprimimos los parámetros, documentación y valores por defecto.
print("LogisticRegression parameters:\n" + lr.explainParams() + "\n")

# Aprendemos un modelo LogisticRegression, utiliza los parámetros almacenados en lr.
model1 = lr.fit(training)

LogisticRegression parameters:
aggregationDepth: suggested depth for treeAggregate (>= 2). (default: 2)
elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0)
family: The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial (default: auto)
featuresCol: features column name. (default: features)
fitIntercept: whether to fit an intercept term. (default: True)
labelCol: label column name. (default: label)
lowerBoundsOnCoefficients: The lower bounds on coefficients if fitting under bound constrained optimization. The bound matrix must be compatible with the shape (1, number of features) for binomial regression, or (number of classes, number of features) for multinomial regression. (undefined)
lowerBoundsOnIntercepts: The lower bounds on intercepts if fitting under bound constrained optimization. The bou

21/11/18 10:16:21 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
21/11/18 10:16:21 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS


## Otra forma de especificar los parámetros

In [5]:
# También podemos especificar los parámetros usando un dictionary de Python como paramMap.
paramMap = {lr.maxIter: 20}
paramMap[lr.maxIter] = 30  # Especificamos un parametro, sobrescribiendo el maxIter original.
paramMap.update({lr.regParam: 0.1, lr.threshold: 0.55})   # Especificamos múltiples parámetros a la vez.

# Podemos combinar paramMaps, son diccionarios de Python
paramMap2 = {lr.probabilityCol: "myProbability"}  # Cambiamos el nombre de la columna de salida
paramMapCombined = paramMap.copy()
paramMapCombined.update(paramMap2)

# Aprendemos un nuevo modelo usando los parámetros en paramMapCombined.
# paramMapCombined sobrescribe todos los parámetros establecidos anteriormente con lr.set*
model2 = lr.fit(training, paramMapCombined)

## Obtenemos los resultados en test

In [6]:
# Preparamos el conjunto de test
test = spark.createDataFrame([
    (1.0, Vectors.dense([-1.0, 1.5, 1.3])),
    (0.0, Vectors.dense([3.0, 2.0, -0.1])),
    (1.0, Vectors.dense([0.0, 2.2, -1.5]))], ["label", "features"])

# Obtenemos las predicciones sobre los datos de test usando Transformer.transform()
# LogisticRegression.transform solo usa la columna llamada features
# El método model2.transform() devuelve una columna "myProbability" column en vez de 
# 'probability' ya que hemos cambiado el parámetro lr.probabilityCol 
prediction = model2.transform(test)
selected = prediction.select("features", "label", "myProbability", "prediction")
for row in selected.collect():
    print(row)

Row(features=DenseVector([-1.0, 1.5, 1.3]), label=1.0, myProbability=DenseVector([0.0571, 0.9429]), prediction=1.0)
Row(features=DenseVector([3.0, 2.0, -0.1]), label=0.0, myProbability=DenseVector([0.9239, 0.0761]), prediction=0.0)
Row(features=DenseVector([0.0, 2.2, -1.5]), label=1.0, myProbability=DenseVector([0.1097, 0.8903]), prediction=1.0)


# Ejemplos de código con LogisticRegression: Pipeline
Vamos a entrenar un modelo de regresión logística con MLlib. En este caso, haremos todo el proceso mediante una pipeline, incluyendo la creación de características.

In [7]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer

# Preparamos los documentos de entrenamiento a partir de una lista de tuplas (id, text, label)
training = spark.createDataFrame([
    (0, "a b c d e spark", 1.0),
    (1, "b d", 0.0),
    (2, "spark f g h", 1.0),
    (3, "hadoop mapreduce", 0.0)], ["id", "text", "label"])

# Configuramos una pipeline de ML, que consiste en tres etapas: tokenizer, hashingTF y lr.
tokenizer = Tokenizer(inputCol="text", outputCol="words")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")
lr = LogisticRegression(maxIter=10, regParam=0.01)
pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])

# Entrenamos la pipeline con los documentos
model = pipeline.fit(training)

21/11/18 10:16:36 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
21/11/18 10:16:36 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


## Obtenemos los resultados en test

In [8]:
# Preparamos el conjunto de test con documentos no etiquetados (id, text)
test = spark.createDataFrame([
    (4, "spark i j k"),
    (5, "l m n"),
    (6, "mapreduce spark"),
    (7, "apache hadoop")], ["id", "text"])

# Hacemos predicciones sobre el test
prediction = model.transform(test)
selected = prediction.select("id", "text", "prediction")
for row in selected.collect():
    print(row)

Row(id=4, text='spark i j k', prediction=0.0)
Row(id=5, text='l m n', prediction=0.0)
Row(id=6, text='mapreduce spark', prediction=0.0)
Row(id=7, text='apache hadoop', prediction=0.0)


# Selección de modelos / ajuste de parámetros
Spark también permite llevar acabo la selección de parámetros de los algoritmos mediante dos formas:
* CrossValidation
* Train/Validation split


## Selección de modelos con CrossValidator

In [9]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Preparamos los documentos de entrenamiento etiquetados.
training = spark.createDataFrame([
    (0, "a b c d e spark", 1.0),
    (1, "b d", 0.0),
    (2, "spark f g h", 1.0),
    (3, "hadoop mapreduce", 0.0),
    (4, "b spark who", 1.0),
    (5, "g d a y", 0.0),
    (6, "spark fly", 1.0),
    (7, "was mapreduce", 0.0),
    (8, "e spark program", 1.0),
    (9, "a e c l", 0.0),
    (10, "spark compile", 1.0),
    (11, "hadoop software", 0.0)], 
    ["id", "text", "label"])

# Configuramos la pipeline, que consiste de tres etapas:  tokenizer, hashingTF, and lr.
tokenizer = Tokenizer(inputCol="text", outputCol="words")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")
lr = LogisticRegression(maxIter=10)
pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])

CrossValidator requiere de un **estimador**, un conjunto de **parámetros** sobre los que realizar la búsqueda y un **evaluador**. Además, hay que indicar el **número de particiones** a utilizar.

In [10]:
# Tratamos la Pipeline como un Estimator para incluirla en una instancia de CrossValidator.
# De esta forma podemos optimizar los parámetros de todas las fases de manera conjunta.

# Un CrossValidator require de un Estimator, un conjunto de ParamMaps del Estimator y un Evaluator.
# Hacemos usod del ParamGridBuilder para construir el grid de parámetros.
# Usamos 3 valores para hashingTF.numFeatures y 2 valores para lr.regParam,
# Este grid tendrá 3 x 2 = 6 combinaciones de las que se elegirá el modelo.
paramGrid = ParamGridBuilder() \
    .addGrid(hashingTF.numFeatures, [10, 100, 1000]) \
    .addGrid(lr.regParam, [0.1, 0.01]) \
    .build()

crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(),
                          numFolds=2)  # usar más de 3 particiones en la práctica

# Ejecutamos la validación cruzada, y elegimos el mejor conjunto de parámetros.
cvModel = crossval.fit(training)

In [11]:
# Preparamos los documentos de test sin etiqueta
test = spark.createDataFrame([
    (4, "spark i j k"),
    (5, "l m n"),
    (6, "mapreduce spark"),
    (7, "apache hadoop")], 
    ["id", "text"])

# Realizamos las predicciones sobre los documentos de test. cvModel  utiliza el mejor modelo encontrado.
prediction = cvModel.transform(test)
selected = prediction.select("id", "text", "prediction", "probability")
for row in selected.collect():
    print(row)

Row(id=4, text='spark i j k', prediction=1.0, probability=DenseVector([0.3407, 0.6593]))
Row(id=5, text='l m n', prediction=0.0, probability=DenseVector([0.9432, 0.0568]))
Row(id=6, text='mapreduce spark', prediction=1.0, probability=DenseVector([0.3449, 0.6551]))
Row(id=7, text='apache hadoop', prediction=0.0, probability=DenseVector([0.9563, 0.0437]))


## Selección de modelos con TrainValidationSplit

In [12]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit

# Preparamos los datos de entrenamiento y test.
data = spark.read.format("libsvm")\
    .load("datos/sample_linear_regression_data.txt") # Si falla la lectura copiar a la raíz del disco duro y cambiar ruta

train, test = data.randomSplit([0.7, 0.3])
lr = LinearRegression(maxIter=10, regParam=0.1)

21/11/18 10:17:19 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


TrainValidationSplit requiere de un **estimador**, un conjunto de **parámetros** sobre los que realizar la búsqueda y un **evaluador**. Además, hay que indicar el **tamaño del training** respecto a la validación (trainRatio)

In [13]:
# Usamos un ParamGridBuilder para construir el grid de parámetros sobre el que realizar la búsqueda.
# TrainValidationSplit probará todas las combinaciones de valores para quedarse con la mejor.
paramGrid = ParamGridBuilder()\
    .addGrid(lr.regParam, [0.1, 0.01]) \
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])\
    .build()

# El estimador es la regresión lineal.
# Un TrainValidationSplit requiere de un Estimator, un conjunto de parámetros ParamMaps y un Evaluator.
tvs = TrainValidationSplit(estimator=lr,
                           estimatorParamMaps=paramGrid,
                           evaluator=RegressionEvaluator(),                           
                           trainRatio=0.8)# 80% of the data for training and 20% for validation.

# Ejecutamos TrainValidationSplit y obtenemos la mejor combinación de parámetros.
model = tvs.fit(train)

21/11/18 10:17:21 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [14]:
# Hacemos las predicciones en test, model es el modelo con la mejor combinación de parámetros.
prediction = model.transform(test)
for row in prediction.take(5):
    print(row)

Row(label=-28.571478869743427, features=SparseVector(10, {0: -0.4597, 1: -0.5489, 2: 0.3342, 3: -0.1599, 4: -0.731, 5: 0.1824, 6: -0.4839, 7: 0.0814, 8: -0.8401, 9: -0.8896}), prediction=1.6403414438011534)
Row(label=-28.046018037776633, features=SparseVector(10, {0: 0.9493, 1: 0.3285, 2: 0.7493, 3: -0.0067, 4: 0.2936, 5: 0.0045, 6: 0.5006, 7: 0.3875, 8: 0.607, 9: -0.7946}), prediction=-0.5212646770876991)
Row(label=-23.51088409032297, features=SparseVector(10, {0: -0.4684, 1: 0.147, 2: 0.9114, 3: -0.9838, 4: 0.4506, 5: 0.6456, 6: 0.8265, 7: 0.5627, 8: -0.8299, 9: 0.4069}), prediction=0.4436828624401702)
Row(label=-22.949825936196074, features=SparseVector(10, {0: 0.4798, 1: 0.02, 2: -0.8828, 3: 0.2755, 4: 0.0155, 5: 0.9653, 6: 0.6623, 7: -0.7708, 8: 0.1773, 9: 0.4782}), prediction=1.6603513854001557)
Row(label=-22.837460416919342, features=SparseVector(10, {0: -0.1736, 1: -0.334, 2: 0.9351, 3: -0.6431, 4: -0.1336, 5: -0.4245, 6: -0.4093, 7: -0.9302, 8: 0.47, 9: -0.6231}), prediction=-